# Modeling Sublimation Event Occurrence

Written by: Daniel Hogan

10 April 2025

It's raining outside with a weak AR-like event impacting northern VI

This notebook will develop, test, and validate a few different model strategies to classify when sublimation events occur using ERA5 reanalysis data.

## Imports

In [ ]:
# general
import datetime as dt
import glob
import os
import logging
from typing import Tuple, List

# data 
import xarray as xr 
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib import gridspec
import seaborn as sns

# helper tools
from metpy import calc, units
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch

# Data driven modeling 
from sklearn.model_selection import (
    GroupKFold,
    ShuffleSplit,
    GroupShuffleSplit,
    KFold,
    StratifiedGroupKFold,
    StratifiedKFold,
    StratifiedShuffleSplit,
    TimeSeriesSplit,
    train_test_split,
    cross_val_predict,
    cross_validate,
    cross_val_score,
    ValidationCurveDisplay,
    GridSearchCV,
    RandomizedSearchCV,
    RepeatedStratifiedKFold
)
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier
import optuna

# stats
import shapely.geometry as shp
from itertools import product
from scipy.stats import pearsonr, spearmanr, linregress
from scipy.ndimage import label, find_objects
import pymannkendall as mk
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms


## Data

### Functions

In [ ]:
# Function to load and subset ERA5 data
def load_era5_data(pressure_level=500):
    """
    Load and subset ERA5 data based on latitude and longitude selections.
    """
    ds = xr.open_mfdataset('/storage/dlhogan/data/raw_data/ERA5_reanalysis_western_NA_202*.nc')
    ds = ds.sel(pressure_level=pressure_level)
    return ds

def calc_valley_aligned_wind_speed(wind_speed, wind_dir, valley_alignment):
    """
    Calculate the valley aligned wind speed
    """
    # calculate the angle between the wind direction and the valley alignment
    angle = np.abs(wind_dir - valley_alignment)
    # calculate the valley aligned wind speed
    valley_aligned_wind_speed = wind_speed * np.cos(np.radians(angle))
    return valley_aligned_wind_speed

    # Calculate the gradients at 500-mb
def calc_era5_gradients(ds):
    u = ds['u']          # shape: (time, lat, lon)
    v = ds['v']
    temp = ds['t']
    q = ds['q']          # specific humidity
    zeta = ds['vo']      # relative vorticity
    z = ds['z']          # geopotential height

    # Get spatial resolution (assumes regular grid)
    dy = np.deg2rad(ds.latitude.diff('latitude').mean()) * 6371000  # m per degree latitude
    dx = np.deg2rad(ds.longitude.diff('longitude').mean()) * 6371000 * np.cos(np.deg2rad(ds.latitude.mean()))  # m per degree longitude

    # Convert degrees to meters
    Re = 6.371e6  # Earth radius in meters
    lat_rad = np.deg2rad(ds.latitude.diff('latitude'))

    # Calculate dx and dy in meters
    dx = np.deg2rad(ds.longitude.diff('longitude').mean()) * Re * np.cos(np.deg2rad(ds.latitude.mean())).values
    dy = np.abs(np.deg2rad(ds.latitude.diff('latitude').mean()) * Re).values  # m per degree latitude

    # Compute gradients
    dT_dx = (temp.differentiate('longitude') / dx) * 1000 # Convert to K/km
    dT_dy = (temp.differentiate('latitude') / dy) * 1000 # Convert to K/km

    # Compute specific humidity gradients
    dq_dx = (q.differentiate('longitude') / dx) * 1000 # Convert to g/kg/km
    dq_dy = (q.differentiate('latitude') / dy) * 1000 # Convert to g/kg/km

    # Compute relative vorticity gradient
    dZeta_dx = (zeta.differentiate('longitude') / dx) * 1000 # Convert to 1/s/km
    dZeta_dy = (zeta.differentiate('latitude') / dy) * 1000 # Convert to 1/s/km

    # Optionally compute gradient magnitude and advection term
    advection_T = -(u * dT_dx + v * dT_dy)/1000 # units of K/s
    advection_q = -(u * dq_dx + v * dq_dy)/1000 # units of g/kg/s
    advection_zeta = -(u * dZeta_dx + v * dZeta_dy)/1000 # units of 1/s^2

    # convert these to float32
    advection_T = advection_T.astype(np.float32).squeeze()
    advection_q = advection_q.astype(np.float32).squeeze()
    advection_zeta = advection_zeta.astype(np.float32).squeeze()

    # add the advection terms to the dataset
    ds['advection_T'] = (('valid_time', 'latitude', 'longitude'), advection_T.data)
    ds['advection_q'] = (('valid_time', 'latitude', 'longitude'), advection_q.data)
    ds['advection_zeta'] = (('valid_time', 'latitude', 'longitude'), advection_zeta.data)

    return ds

def subset_ds(ds, winter_slice, calc_gradients=True):
    """
    Subset the dataset to the winter months
    """
    # subset the data   
    ds_subset = ds.sel(valid_time=winter_slice)

    # if calc_gradients is True, calculate the gradients of the variables
    if calc_gradients:
        # call the calc_era5_gradients function to calculate the gradients
        ds_subset = calc_era5_gradients(ds_subset)

    # calculate the wind speed and direction
    ds_subset['wind_speed'] = np.sqrt(ds_subset.u**2 + ds_subset.v**2)
    # create a column with the wind direction
    ds_subset['wind_dir'] = np.arctan2(ds_subset['v'], ds_subset['u']) * 180 / np.pi
    # convert wind direction to compass direction
    ds_subset['wind_dir'] = (270 - ds_subset['wind_dir']) % 360

    # calculate "northerliness"
    ds_subset['northerliness'] = np.cos(np.deg2rad(ds_subset['wind_dir']))
    
    # calculate dew point from relative humidity and temeprature
    # convert temperature to celsius
    ds_subset['t'] = units.units('degC') * (ds_subset['t']-273.15)

    # calculate vpd
    ds_subset['vpd'] = calc.saturation_vapor_pressure((ds_subset['t'])) - ds_subset['q'] * units.units('Pa')
    return ds_subset.squeeze()

def spatial_subset(ds, lat_range, lon_range, point=False):
    """
    Subset the dataset to a specific latitude and longitude range.
    """
    if point:
        # If point is True, select the nearest point in the dataset
        ds_subset = ds.sel(latitude=lat_range, longitude=lon_range, method='nearest')
        return ds_subset
    else:
        # if point is False, select the latitude and longitude range
        # if is a slice, use slice
        if isinstance(lat_range, slice) and isinstance(lon_range, slice):
            ds_subset = ds.sel(latitude=lat_range, longitude=lon_range)
        else:
            # If lat_range and lon_range are lists or tuples, use slice
            lat_range = slice(lat_range[0], lat_range[1])
            lon_range = slice(lon_range[0], lon_range[1])
            ds_subset = ds.sel(latitude=lat_range, longitude=lon_range)
        return ds_subset

### ERA5 data

Spatial selection is performed here!! Very important.

In [ ]:
# Load the ERA5 data
ds = load_era5_data()

# Define the winter slices for 2022 and 2023
winter_2022 = slice('2021-12-01T00:00:00', '2022-03-31T23:00:00')
winter_2023 = slice('2022-12-01T00:00:00', '2023-03-31T23:00:00')

# Subset the dataset for the winter months and calculate gradients
ds_all_winter_2022 = subset_ds(ds, winter_2022, calc_gradients=True)
ds_all_winter_2023 = subset_ds(ds, winter_2023, calc_gradients=True)

# close the dataset to free up memory
try:
    ds.close()
    ds = None
except Exception as e:
    print(f"Dataset already closed or error closing: {e}")

In [ ]:
# geographic data and orietation of the valley
valley_orientation = 305
cross_valley_orientation = 215
# get the coordingates of Grand Junction, CO
gj = shp.Point(-108.5506, 39.0639)
# get the coordinates of Gothic, CO
gothic = shp.Point(-106.9898, 38.9592)
line = shp.LineString([gj, gothic])

# Define the latitude and longitude slices for the region of interest region
lat_sel_5_by_5 = slice(39.5, 38.25)
lon_sel_5_by_5 = slice(-107.5, -106.5)

lat_sel_10_by_10 = slice(40, 37.75)
lon_sel_10_by_10 = slice(-108.75,-106.5)

lat_sel_15_by_15 = slice(40.5, 37)
lon_sel_15_by_15 = slice(-110, -106.5)


# Single grid-cell selection for Gothic, CO
ds_gt_winter_2022 = spatial_subset(ds_all_winter_2022, gothic.y, gothic.x,  point=True)
ds_gt_winter_2023 = spatial_subset(ds_all_winter_2023, gothic.y, gothic.x, point=True)

# 5 by 5 degree selection
ds_5_by_5_winter_2022 = spatial_subset(ds_all_winter_2022, lat_sel_5_by_5, lon_sel_5_by_5).mean(dim=['latitude', 'longitude'])
ds_5_by_5_winter_2023 = spatial_subset(ds_all_winter_2023, lat_sel_5_by_5, lon_sel_5_by_5).mean(dim=['latitude', 'longitude'])

# 10 by 10 degree selection
ds_10_by_10_winter_2022 = spatial_subset(ds_all_winter_2022, lat_sel_10_by_10, lon_sel_10_by_10).mean(dim=['latitude', 'longitude'])
ds_10_by_10_winter_2023 = spatial_subset(ds_all_winter_2023, lat_sel_10_by_10, lon_sel_10_by_10).mean(dim=['latitude', 'longitude'])

# 15 by 15 degree selection
ds_15_by_15_winter_2022 = spatial_subset(ds_all_winter_2022, lat_sel_15_by_15, lon_sel_15_by_15).mean(dim=['latitude', 'longitude'])
ds_15_by_15_winter_2023 = spatial_subset(ds_all_winter_2023, lat_sel_15_by_15, lon_sel_15_by_15).mean(dim=['latitude', 'longitude'])


In [ ]:
# close the datasets to free up memory
try:
    ds_all_winter_2022.close()
    ds_all_winter_2022 = None
except Exception as e:
    print(f"Dataset already closed or error closing: {e}")

try:
    ds_all_winter_2023.close()
    ds_all_winter_2023 = None
except Exception as e:
    print(f"Dataset already closed or error closing: {e}")

### Latent heat flux data

In [ ]:
# take in sublimation data
latent_heat_flux_2022 = pd.read_csv('./01_data/processed_data/sublimation/w22_latent_heat_flux_splash_ap.csv', index_col=0, parse_dates=True)
latent_heat_flux_2023 = pd.read_csv('./01_data/processed_data/sublimation/w23_latent_heat_flux_3m.csv', index_col=0, parse_dates=True)

### Event classifications

In [ ]:
# open large and short events dataset 
w22_large_sublimation_long_events = pd.read_csv('./01_data/processed_data/sublimation/w22_large_sublimation_long_events.csv', index_col=0, parse_dates=True)
w22_large_sublimation_short_events = pd.read_csv('./01_data/processed_data/sublimation/w22_large_sublimation_spiky_events.csv', index_col=0, parse_dates=True)
w23_large_sublimation_long_events = pd.read_csv('./01_data/processed_data/sublimation/w23_large_sublimation_long_events.csv', index_col=0, parse_dates=True)
w23_large_sublimation_short_events = pd.read_csv('./01_data/processed_data/sublimation/w23_large_sublimation_spiky_events.csv', index_col=0, parse_dates=True)

### Precipitation data (from Gothic weighing bucket)

In [8]:
# open precipitation data
w22_precipitation = pd.read_csv('./01_data/processed_data/precipitation/w22_all_precip_30min.csv', index_col=0, parse_dates=True)['SAIL_gts_pluvio'].loc['2021-12-01':'2022-03-31']
w23_precipitation = pd.read_csv('./01_data/processed_data/precipitation/w23_all_precip_30min.csv', index_col=0, parse_dates=True)['SAIL_gts_pluvio'].loc['2022-12-01':'2023-03-31']

### Long term observations

#### GJT sounding 500-mb winds

In [9]:
# observations from GUC sounding
guc_obs = pd.read_csv('./01_data/processed_data/winter_500mb_GUC_winds.csv', index_col=0)

#### Long-term Dec-Mar precipitation from billy barr

In [10]:
# NEED TO GET THIS!!

## Data Processing

### Functions

In [11]:
def compute_cross_correlation(df, target_var, lag_range=(-10, 10)):
    """
    Compute cross-correlations between a target variable and other variables with different time lags.

    Parameters:
    - df: pandas DataFrame with time-indexed meteorological variables.
    - target_var: Name of the target variable (e.g., "sublimation").
    - lag_range: Tuple specifying min and max lag (e.g., (-10, 10)).

    Returns:
    - A DataFrame containing cross-correlations for each lag.
    """
    variables = df.columns.drop(target_var)  # Exclude the target variable itself
    lag_values = range(lag_range[0], lag_range[1] + 1)

    correlation_results = pd.DataFrame(index=lag_values, columns=variables)

    for lag in lag_values:
        shifted_target = df[target_var].shift(lag)  # Shift target variable
        correlation_results.loc[lag] = df[variables].corrwith(shifted_target)

    return correlation_results.astype(float)

def find_best_synoptic_variable(surface_obs, synoptic_data, method='pearson'):
    """
    Finds the synoptic-scale variable that is most closely related to a given surface meteorology observation.

    Parameters:
    - surface_obs (pd.Series): Time series of the surface meteorology variable (e.g., temperature, humidity).
    - synoptic_data (pd.DataFrame): Time series of synoptic-scale variables (e.g., geopotential height, wind speed).
    - method (str): Correlation method ('pearson' for linear, 'spearman' for rank-based). Default is 'pearson'.

    Returns:
    - dict: Contains the best-correlated variable, its correlation coefficient, and the full correlation results.
    """
    
    correlations = {}

    for var in synoptic_data.columns:
        synoptic_var = synoptic_data[var]

        # Drop NaN values
        valid_idx = surface_obs.index.intersection(synoptic_var.dropna().index)
        x = surface_obs.loc[valid_idx]
        y = synoptic_var.loc[valid_idx]

        if len(x) < 10:  # Require a minimum number of valid data points
            continue

        # Compute correlation
        if method == 'pearson':
            corr, _ = pearsonr(x, y)
        elif method == 'spearman':
            corr, _ = spearmanr(x, y)
        else:
            raise ValueError("Invalid method. Choose 'pearson' or 'spearman'.")

        correlations[var] = corr

    # Find the variable with the highest absolute correlation
    best_var = max(correlations, key=lambda k: abs(correlations[k]))
    
    return {
        'best_variable': best_var,
        'best_correlation': correlations[best_var],
        'all_correlations': correlations
    }

def select_data(lh_df, ppt_data, reanalysis_ds, short_events, long_events):
    """
    This function will provide the necessary data for the analysis of sublimation events in relation to the reanalysis data.
    """
    # resample latent heat flux data and precipitation data to 3H
    lh_df_3H = lh_df.resample('3H').mean()
    ppt_data_3H = ppt_data.resample('3H').sum()
    #   filter out exrtreme values
    lh_df_3H = lh_df_3H.where(lh_df_3H >-50, np.nan)
    # select the reanalysis pressure level
    ds = reanalysis_ds.resample(valid_time='3H').mean()

    return lh_df_3H, ppt_data_3H, ds, short_events, long_events

# Plotting 
def plot_surface_and_reanalysis_vars(lh_df, ppt_data, era5_ds, short_events, long_events, lh_var, reanalysis_var1, reanalysis_var2):
    """
    This function will plot the relationship between the latent heat flux and the reanalysis variables.
    Inputs:
    lh_df: DataFrame with the latent heat flux data
    ppt_data: DataFrame with the precipitation data
    era5_ds: xarray dataset with the reanalysis data
    short_events: DataFrame with the short, intense sublimation events
    long_events: DataFrame with the long sublimation events
    lh_var: string with the name of the latent heat flux variable
    reanalysis_var1: string with the name of the first reanalysis variable
    reanalysis_var2: string with the name of the second reanalysis variable
    """
    lh_df, ppt_data, era5_ds, short_events, long_events = select_data(lh_df, ppt_data, era5_ds, short_events, long_events)

    xlabel_dict = {'u': 'Wind Speed (m/s)', 
                   'v': 'Wind Speed (m/s)', 
                   't': 'Temperature (C)', 
                   'q': 'Specifc Humidity (kg/kg)', 
                   'wind_speed': 'Wind Speed (m/s)', 
                   'wind_dir': 'Wind Direction (degrees)',
                   'northerliness': 'Index of how northerly wind is (+1 strong north wind, -1 strong south wind)',
                   }
    xlims_dict = {
             'q': (0, 0.0035),
             't': (-40, -5),}
             
    fig, axs = plt.subplots(ncols=2, figsize=(10, 5))
    ax = axs[0]
    ax.scatter(era5_ds[reanalysis_var1], lh_df[lh_var])
    # ax.set_xlabel('Wind Speed')
    ax.set_ylabel('Daily Average Latent Heat Flux (W/m^2)')
    if reanalysis_var1 in xlims_dict.keys():
            ax.set_xlim(xlims_dict[reanalysis_var1])

    # add edge color around the large sublimation events 
    ax.scatter(era5_ds.sel(valid_time=long_events.index)[reanalysis_var1], lh_df.loc[long_events.index][lh_var], c="black", edgecolors='black')
    ax.scatter(era5_ds.sel(valid_time=ppt_data[ppt_data>1].index)[reanalysis_var1], lh_df.loc[ppt_data[ppt_data>1].index][lh_var], c="red", edgecolors='red', label='Precipitation')
    ax.set_xlabel(xlabel_dict[reanalysis_var1])
    ax.axvline(28, color='black', linestyle='--')
    
    ax = axs[1]
    ax.scatter(era5_ds[reanalysis_var2], lh_df[lh_var])
    ax.scatter(era5_ds.sel(valid_time=long_events.index)[reanalysis_var2], lh_df.loc[long_events.index][lh_var], c="black", edgecolors='black', label='Long Events')
    ax.axvline(33, color='black', linestyle='--')
    ax.scatter(era5_ds.sel(valid_time=ppt_data[ppt_data>1].index)[reanalysis_var2], lh_df.loc[ppt_data[ppt_data>1].index][lh_var], c="red", edgecolors='red', label='Precipitation')
    ax.set_xlabel(xlabel_dict[reanalysis_var2])
    ax.set_ylabel('Daily Average Latent Heat Flux (W/m^2)')
    ax.legend()
    if reanalysis_var2 in xlims_dict.keys():
            ax.set_xlim(xlims_dict[reanalysis_var2])
    # add colorbar
    # plt.colorbar(ax.collections[0], ax=axs, orientation='horizontal', label='500 mb Temperature (C)')

    for ax in axs:
        ax.set_ylim(-10, lh_df[lh_var].max()+10)
    return fig

In [140]:
def add_events_col(df, 
                   short_events, 
                   long_events):
    """
    This function will add a column to the dataframe indicating whether the event is a short or long event.
    """
    # add event column, but only for the long sublimation events
    df['event'] = 0
    if long_events is not None:
        df.loc[long_events, 'event'] = 1
    else:
        print("No long events provided.")
    if short_events is not None:
        df.loc[short_events, 'event'] = 1
    else:
        print("No short events provided.")
    return df

def create_df_combined(ds1, ds2, variables, resample=True, vars_to_add=None):
    # merge datasets
    ds_combined = xr.merge([ds1, ds2])

    # convert to dataframe
    df_combined = ds_combined[variables].to_dataframe()[variables]

    # if vars_to_add is not None, add the key as the name and the value as the value
    if vars_to_add is not None:
        for key, value in vars_to_add.items():
            if key == "latent_heat_flux":
                # if the value is a dataframe, resample to 3H and take the mean
                if isinstance(value, pd.DataFrame):
                    df_combined = pd.merge(value, df_combined, left_index=True, right_index=True)
                else:
                    print(f"{key} is not a dataframe. Make sure it is.")
            else:
                df_combined[key] = value
    # if resample is True, resample to 3H and take the mean
    if isinstance(resample, bool) and resample:
        df_combined = df_combined.resample('3H').mean()
        # add event column
        df_combined = add_events_col(df_combined, 
                                     short_events=None, 
                                     long_events=pd.concat([w22_large_sublimation_long_events,
                                                            w23_large_sublimation_long_events[:-1]]).index)
    elif isinstance(resample, str):
        df_combined = df_combined.resample('3H').mean()
        df_combined = add_events_col(df_combined, 
                                     short_events=None, 
                                     long_events=pd.concat([w22_large_sublimation_long_events,
                                                            w23_large_sublimation_long_events[:-1]]).index)
        df_combined = df_combined.resample(resample).agg({'sublimation': 'sum', 
                                                            'u': 'mean', 
                                                            'v': 'mean', 
                                                            't': 'mean',
                                                            'q' : 'min',
                                                            'vpd': 'min',
                                                            'wind_speed': 'max', 
                                                            'precipitation':'sum',
                                                            'advection_T': 'mean',
                                                            'advection_q': 'mean',
                                                            'advection_zeta': 'mean', 
                                                            'event': 'max'})
    # reset the index and dropna
    df_combined = df_combined.reset_index()
    # if event is in the columns
    if 'event' in df_combined.columns:
        # replace event values equal to 0 with "non-event" and 1 with "event"
        df_combined['event'] = df_combined['event'].replace({0: 'non-event', 1: 'event'})
    else:
        print("No event column found.")
     
    return df_combined


### Latent Heat Data

In [37]:
# 2022
if 'splash_ap' in latent_heat_flux_2022.columns:
    latent_heat_flux_2022.rename(columns={'splash_ap': 'sublimation'}, inplace=True)
# 2023
if 'sos_3m' in latent_heat_flux_2023.columns:
    latent_heat_flux_2023.rename(columns={'sos_3m': 'sublimation'}, inplace=True)
# combine the latent heat flux data
latent_heat_flux = pd.concat([latent_heat_flux_2022, latent_heat_flux_2023], axis=0)

### Precipitation

In [38]:
# add precipitation
precip_df = pd.concat([w22_precipitation, w23_precipitation], axis=0).resample('3H').sum()

### Combine Datasets

In [39]:
selected_datasets = [ds_5_by_5_winter_2022, ds_5_by_5_winter_2023]
variables = ['t', 'wind_speed','vpd', 'u','v','q','advection_T', 'advection_q', 'advection_zeta']
# merge the two datasets
combined_df = create_combined_df(selected_datasets[0], 
                                 selected_datasets[1], 
                                 variables, 
                                 resample=True, 
                                 vars_to_add={'precipitation': precip_df,
                                              'latent_heat_flux': latent_heat_flux})

In [42]:
combined_df_12H = create_combined_df(selected_datasets[0],
                                      selected_datasets[1], 
                                      variables, 
                                      resample='12H', 
                                      vars_to_add={'precipitation': precip_df,
                                              'latent_heat_flux': latent_heat_flux})

### Basic Statistics

#### Calculate correlations between variables
*Note: Can adjust easily for 2023, but we see similar results.*

In [43]:
# Here, we use just the ERA5 grid-cell directly over Gothic
lh_df, ppt_data, era5_ds, short_events, long_events = select_data(latent_heat_flux_2022, 
                                                                  w22_precipitation, 
                                                                  ds_5_by_5_winter_2022, 
                                                                  w22_large_sublimation_short_events, 
                                                                  w22_large_sublimation_long_events,)

In [44]:
# largest_events = (long_events.groupby('event').mean() > 15)
synoptic_data = era5_ds.to_dataframe()[variables].loc[[t[0] for t in long_events.groupby('event').idxmax().values]]
surface_obs = long_events.groupby('event').max().set_index(synoptic_data.index).splash_ap

result = find_best_synoptic_variable(surface_obs, synoptic_data, method='spearman')

print("Best correlated synoptic variable:", result['best_variable'])
for var, corr in result['all_correlations'].items():
    print(f"{var}: {corr:.2f}")

Calculate correlations between variables using average of all grids

In [20]:
# Calculate correlations between variables using average of all grids
lh_df, ppt_data, era5_ds, short_events, long_events = select_data(latent_heat_flux_2022, 
                                                                  w22_precipitation, 
                                                                  ds_gt_winter_2022, 
                                                                  w22_large_sublimation_short_events, 
                                                                  w22_large_sublimation_long_events,)

In [21]:
# largest_events = (long_events.groupby('event').mean() > 15)
synoptic_data = era5_ds.to_dataframe()[variables].loc[[t[0] for t in long_events.groupby('event').idxmax().values]]
surface_obs = long_events.groupby('event').max().set_index(synoptic_data.index).splash_ap

result = find_best_synoptic_variable(surface_obs, synoptic_data, method='spearman')

print("Best correlated synoptic variable:", result['best_variable'])
for var, corr in result['all_correlations'].items():
    print(f"{var}: {corr:.2f}")

Discussion:

We averaged the synoptic level (500 mb) data in two differnt ways: the average of a 10x10 ERA5 grid centered over the East River valley and the nearest grid cell over Gothic. We took this data and calculated spearman correlations with the maximum surface latent heat flux observation during long events. 

We found that both the single grid cell and the regional average both correlate well with u-wind and wind-speed. However, large differences are observed for temperature (> for single grid cell), vorticity (> for grid-average)

This likely means that including certain variables may not be super helpful. This is only for the 2023 data.

We get a pretty different story during 2022. For the gridded average, v, q, and t have the largest relationships, with wind speed being much lower. This might be why including these other variables prove to be useful within the model. It seems q, t, u, v, and vo are the most useful. 

#### Percentile rank figures

In [22]:
# Calculate percentiles
percentiles = combined_df.rank(method="average", pct=True)[['t', 'wind_speed','vpd', 'u','v','q','advection_T', 'advection_q', 'advection_zeta','sublimation']]

# Subset for different sublimation thresholds
percentiles_sub_50 = percentiles[percentiles['sublimation'] > 0.75]
percentiles_sub_75 = percentiles[percentiles['sublimation'] > 0.85]
percentiles_sub_95 = percentiles[percentiles['sublimation'] > 0.95]

# Drop 'sublimation' since it's the target variable
variables = ['t', 'wind_speed','vpd', 'u','v','q','advection_T', 'advection_q', 'advection_zeta']

# Create a grouped DataFrame for boxplot
melted_50 = percentiles_sub_50[variables].melt(var_name="Variable", value_name="Percentile")
melted_50["Threshold"] = "> 70%"

melted_75 = percentiles_sub_75[variables].melt(var_name="Variable", value_name="Percentile")
melted_75["Threshold"] = "> 85%"

melted_95 = percentiles_sub_95[variables].melt(var_name="Variable", value_name="Percentile")
melted_95["Threshold"] = "> 95%"

# Combine into a single dataframe
plot_data = pd.concat([melted_50, melted_75, melted_95])

# Plot using seaborn
plt.figure(figsize=(12, 6))
sns.boxplot(x="Variable", y="Percentile", hue="Threshold", data=plot_data, palette=["lightblue", "orange", "red"])

# Customization
plt.title("Boxplot of Meteorological Variables for Different Sublimation Thresholds")
plt.xlabel("Meteorological Variable")
plt.ylabel("Percentile Rank")
plt.legend(title="Sublimation Threshold", loc='upper center')
plt.xticks(rotation=30)  # Rotate x labels for better readability

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


In [214]:
# Calculate percentiles
percentiles = combined_df.rank(method="average", pct=True)[['t', 'wind_speed','vpd', 'u','v','q','advection_T', 'advection_q', 'advection_zeta','sublimation']]

# Subset for different sublimation thresholds
percentiles_event = percentiles[combined_df['event']=='event']
percentiles_non_event = percentiles[combined_df['event']=='non-event']

# Drop 'sublimation' since it's the target variable
variables = ['t', 'wind_speed','vpd', 'u','v','q','advection_T', 'advection_q', 'advection_zeta']

# Create a grouped DataFrame for boxplot
melted_non_event = percentiles_non_event[variables].melt(var_name="Variable", value_name="Percentile")
melted_non_event["Threshold"] = "non-event"

melted_event = percentiles_event[variables].melt(var_name="Variable", value_name="Percentile")
melted_event["Threshold"] = "event"

# Combine into a single dataframe
plot_data = pd.concat([melted_non_event, melted_event]).reset_index(drop=True)

# Plot using seaborn
plt.figure(figsize=(12, 6))
sns.boxplot(x="Variable", y="Percentile", hue="Threshold", data=plot_data, palette=["orange", "red"])

# Customization
plt.title("Boxplot of Meteorological Variables for Different Sublimation Thresholds")
plt.xlabel("Meteorological Variable")
plt.ylabel("Percentile Rank")
plt.legend(title="Sublimation Threshold")
plt.xticks(rotation=15)  # Rotate x labels for better readability

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


#### Temporal cross-correlation figures
When are variables cross correlated with high sublimation rates?

In [23]:
# Example usage
cross_corrs = compute_cross_correlation(combined_df[['t', 'wind_speed','vpd', 'u','v','q','advection_T', 'advection_q', 'advection_zeta','sublimation']], 
                                        target_var='sublimation', 
                                        lag_range=(-12, 12))

# Plot correlation results
plt.figure(figsize=(10, 6))
for var in cross_corrs.columns:
    plt.plot(cross_corrs.index, cross_corrs[var], label=var, marker='o', linestyle='-')

plt.axhline(0, color='black', linestyle='--', linewidth=0.8)  # Reference line at 0
# change out the x-axis for hours by multiplying by 3
plt.xticks(np.arange(-12, 13, 1), labels=np.arange(-12, 13, 1)*3)
plt.xlabel("Time Lag (Hours)")
plt.ylabel("Correlation Coefficient")
plt.title("Cross-Correlation Between Reanalysis Variables and Sublimation")
plt.legend(title="Variables")
plt.grid(alpha=0.3)
plt.ylim(-0.5,0.5)
plt.show()


In [24]:
# Example usage
cross_corrs = compute_cross_correlation(combined_df_12H[['t', 'wind_speed','vpd', 'u','v','q','advection_T', 'advection_q', 'advection_zeta','sublimation']], 
                                        target_var='sublimation', 
                                        lag_range=(-6, 6))

# Plot correlation results
plt.figure(figsize=(10, 6))
for var in cross_corrs.columns:
    plt.plot(cross_corrs.index, cross_corrs[var], label=var, marker='o', linestyle='-')

plt.axhline(0, color='black', linestyle='--', linewidth=0.8)  # Reference line at 0
# change out the x-axis for hours by multiplying by 3
plt.xticks(np.arange(-6, 7, 1), labels=np.arange(-6, 7, 1)/2)
plt.xlabel("Time Lag (Days)")
plt.ylabel("Correlation Coefficient")
plt.title("Cross-Correlation Between Reanalysis Variables and Sublimation")
plt.legend(title="Variables")
plt.grid(alpha=0.3)
plt.ylim(-0.5,0.5)
plt.show()


#### Histograms of each variable

In [25]:
vars_to_plot = ['t', 'wind_speed','vpd', 'u','v','q','advection_T', 'advection_q', 'advection_zeta','sublimation']

# plot histograms of each variable in vars_to_plot
fig, axs = plt.subplots(5, 2, figsize=(12, 12))
for ax, var in zip(axs.flat, vars_to_plot):
    sns.histplot(combined_df[var], ax=ax, kde=True)
    ax.set_title(f"Histogram of {var}")
    ax.set_xlabel(var)
    ax.set_ylabel("Frequency")
plt.tight_layout()

#### Event frequency relative to all 3H observations

In [218]:
# create distribution of events and non-events
fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(combined_df['event'].value_counts().index, combined_df['event'].value_counts().values, color=['blue', 'red'], alpha=0.7)
# add percentage of events and non-events
for i, count in enumerate(combined_df['event'].value_counts().values):
    ax.text(i, count, f"{count} ({count/combined_df.shape[0]*100:.2f}%)", ha='center', va='bottom')
ax.set_title("Distribution of Events and Non-Events")
ax.set_xlabel("Event")
ax.set_ylabel("Frequency")
ax.set_xticklabels(["Non-Event", "Event"])

## Modeling

In [17]:
# Set up logging to save results in the specified directory
log_dir = '/home/dlhogan/GitHub/Synoptic-Sublimation/03_results/model_logs'
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, 'model_results.log')
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')

### Functions

In [75]:
# ------------------------------------------------------------------
# split helper ------------------------------------------------------
# ------------------------------------------------------------------
def split_train_test(data_X, data_y, test_size=0.2, random_state=42):
    """Split data into train and test sets."""
    return train_test_split(data_X, data_y,
                            test_size=test_size,
                            random_state=random_state)

# ------------------------------------------------------------------
# main training / evaluation routine -------------------------------
# ------------------------------------------------------------------
def train_and_evaluate_model(model, model_name,
                             X_train, y_train, X_val, y_val,
                             param_grid=None, search_method="grid",
                             scoring=None, cv=None):
    """
    Train `model`, optionally tune hyper‑parameters, and visualise:
      • confusion matrix          (always)
      • feature importance barplot(if supported)
    """
    # ---------- defaults ------------------------------------------
    if scoring is None:
        scoring = {"F1": "f1",
                   "Average_Precision": metrics.make_scorer(metrics.average_precision_score)}
    if cv is None:
        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    best_params, best_score = None, None

    # ---------- hyper‑parameter search ---------------------------
    if search_method in {"grid", "random"} and param_grid:
        search_cls = GridSearchCV if search_method == "grid" else RandomizedSearchCV
        search = search_cls(estimator=model,
                            param_grid=param_grid if search_method == "grid" else None,
                            param_distributions=param_grid if search_method == "random" else None,
                            scoring=scoring,
                            refit="F1",
                            cv=cv,
                            n_jobs=4,
                            random_state=42 if search_method == "random" else None,
                            return_train_score=True)
        search.fit(X_train, y_train)
        model        = search.best_estimator_
        best_params  = search.best_params_
        best_score   = search.best_score_
        logging.info(f"Best params: {best_params}")
        logging.info(f"Best CV F1 : {best_score:.4f}")

    elif search_method == "optuna":
        def objective(trial):
            # ---- suggest params by model type --------------------
            if isinstance(model, LogisticRegression):
                C = trial.suggest_float("C", 1e-4, 1e2, log=True)
                p = "l2" # trial.suggest_categorical("penalty", ["l1", "l2"])
                if p == "l1":
                    s = "liblinear"
                elif p == "l2":
                    s = "lbfgs"
                clf = LogisticRegression(C=C, penalty=p, solver=s, max_iter=1000)
            elif isinstance(model, RandomForestClassifier):
                clf = RandomForestClassifier(
                    n_estimators     = trial.suggest_int("n_estimators", 30, 150),
                    max_depth        = trial.suggest_int("max_depth", 3, 15),
                    min_samples_split= trial.suggest_int("min_samples_split", 2, 20),
                    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20),
                    max_features     = trial.suggest_categorical("max_features",
                                                                 ["sqrt", "log2", 3, 4]),
                    n_jobs=12)
            elif isinstance(model, XGBClassifier):
                clf = XGBClassifier(
                        max_depth        = trial.suggest_int("max_depth", 4, 8),
                        learning_rate    = trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
                        n_estimators     = trial.suggest_int("n_estimators", 30, 150),
                        subsample        = trial.suggest_float("subsample", 0.6, 1.0),
                        colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0),
                        scale_pos_weight = trial.suggest_float("scale_pos_weight", 5, 15),  # around imbalance ratio
                        reg_alpha        = trial.suggest_float("reg_alpha", 0, 0.5),
                        reg_lambda       = trial.suggest_float("reg_lambda", 0.5, 2.0),
                        # use_label_encoder=False,
                        eval_metric="logloss",
                        n_jobs=12)
            else:
                raise ValueError("Optuna not configured for this model type")

            return cross_val_score(clf, X_train, y_train,
                                   cv=cv, scoring="f1").mean()

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=20)

        best_params = study.best_params
        best_score  = study.best_value
        logging.info(f"Best params (Optuna): {best_params}")
        logging.info(f"Best CV F1 (Optuna): {best_score:.4f}")

        # rebuild the model with best parameters
        model.set_params(**best_params)
        model.fit(X_train, y_train)

    else:
        model.fit(X_train, y_train)

    # ---------- validation evaluation ---------------------------
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    y_pred       = model.predict(X_val)

    f1_score_val      = metrics.f1_score(y_val, y_pred)
    average_precision = metrics.average_precision_score(y_val, y_pred_proba)
    baseline_precision= y_val.mean()
    roc_auc_val       = metrics.roc_auc_score(y_val, y_pred_proba)

    logging.info(f"{model_name} | F1: {f1_score_val:.4f}  AP: {average_precision:.4f}  "
                 f"ROC‑AUC: {roc_auc_val:.4f}")

    # ---------- visualisations ----------------------------------
    ConfusionMatrixDisplay.from_estimator(model, X_val, y_val,
                                          cmap="Blues", colorbar=False,
                                          display_labels=["non‑event", "event"])
    plt.title(f"{model_name} – Confusion Matrix")
    plt.show()

    # ---- feature importance (if available) ---------------------
    # tree models: feature_importances_ ; logistic: coef_
    importances = None
    if hasattr(model, "feature_importances_"):
        importances = model.feature_importances_
    elif hasattr(model, "coef_"):
        importances = np.abs(model.coef_.ravel())  # magnitude for LR

    if importances is not None:
        # obtain feature names or fallback to generic
        if hasattr(X_train, "columns"):
            feature_names = X_train.columns
        else:
            feature_names = [f"f{i}" for i in range(X_train.shape[1])]

        # sort by importance
        idx_sorted = np.argsort(importances)[::-1]
        top_n = min(20, len(idx_sorted))           # show at most 20 for clarity
        idx_sorted = idx_sorted[:top_n]

        plt.figure(figsize=(8, 6))
        plt.barh(range(top_n), importances[idx_sorted][::-1])
        plt.yticks(range(top_n), np.array(feature_names)[idx_sorted][::-1])
        plt.xlabel("Importance")
        plt.title(f"{model_name} – Top {top_n} Feature Importances")
        plt.tight_layout()
        plt.show()

    # ---------- return summary ----------------------------------
    return {"model":               model,
            "best_params":         best_params,
            "best_score":          best_score,
            "f1_score":            f1_score_val,
            "average_precision":   average_precision,
            "baseline_precision":  baseline_precision,
            "roc_auc":             roc_auc_val}


### Train/Test/Validation Split

Very important, check test size

In [91]:
test_size = 0.1
# Establish model select variables that will be used to train the model.
selected_datasets = [ds_10_by_10_winter_2022, 
                     ds_10_by_10_winter_2023]
model_variables = ['t', 'wind_speed','vpd', 'u','v','q','advection_T', 'advection_q', 'advection_zeta']
# merge the two datasets
combined_df = create_combined_df(selected_datasets[0], 
                                 selected_datasets[1], 
                                 variables, 
                                 resample=True, 
                                 vars_to_add={'precipitation': precip_df,
                                              'latent_heat_flux': latent_heat_flux})

data_X = combined_df.dropna(how='any')[model_variables]
data_y = combined_df.dropna(how='any')['event'].replace({'non-event': 0, 'event': 1})    

X_train, X_val, y_train, y_val = split_train_test(data_X, data_y, test_size=test_size, random_state=42)

#### Setup for single variable logistic regression.

In [92]:
# Establish model select variables that will be used to train the model.
single_model_variables = ['wind_speed']

data_X_single = combined_df.dropna(how='any')[single_model_variables]
data_y_single = combined_df.dropna(how='any')['event'].replace({'non-event': 0, 'event': 1})    

X_train_single, X_val_single, y_train_single, y_val_single = split_train_test(data_X_single, data_y_single, test_size=test_size, random_state=42)

### Training and Cross-Validation
Ceate cross-validation object consisting of 10 folds using stratified sampling and shufffle given that this is a binary classification problem. A variation of k-fold which returns stratified folds: each set contains approximately the same percentage of samples of each target class as the complete set. I am shuffling only once to make sure that time series of events does not influence the model result.

### Run the models

In [93]:
# Define models and parameter grids
models = {
    "Logistic Regression (Single)": {
        "model": LogisticRegression(class_weight='balanced', random_state=42),
        "param_grid": {
            "C": [0.01, 0.1, 1, 10],
            "penalty": ["l1", "l2"],
            "solver": ["liblinear", "saga"]
        }
    },
    "Logistic Regression": {
        "model": LogisticRegression(class_weight='balanced', random_state=42),
        "param_grid": {
            "C": [0.01, 0.1, 1, 10],
            "penalty": ["l1", "l2"],
            "solver": ["liblinear", "saga"]
        }
    },
    "Random Forest": {
        "model": RandomForestClassifier(random_state=42, class_weight='balanced'),
        "param_grid": {
            "n_estimators": [50, 100, 150],
            "max_depth": [None, 10, 20],
            "min_samples_split": [2, 5, 10]
        }
    },
    "XGBoost": {
        "model": XGBClassifier(random_state=42, objective="binary:logistic"),
        "param_grid": {
            "n_estimators": [40, 50, 60],
            "scale_pos_weight": [10, 20, 30],
            "learning_rate": [0.1, 0.2, 0.3],
            "max_depth": [5, 7, 10],
            "subsample": [0.5, 0.7, 0.9]
        }
    },
}

# Cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {"F1": "f1", "Average_Precision": metrics.make_scorer(metrics.average_precision_score)}

# Train and evaluate each model
results = {}
for model_name, config in models.items():
    if model_name == "Logistic Regression (Single)":
        results[model_name] = train_and_evaluate_model(
            model=config["model"],
            model_name=model_name,
            X_train=X_train_single,
            y_train=y_train_single,
            X_val=X_val_single,
            y_val=y_val_single,
            param_grid=None, # set to None to use Optuna
            search_method="optuna",  # Change to "random" for RandomizedSearchCV
            scoring=scoring,
            cv=cv
        ) 
    else:
        results[model_name] = train_and_evaluate_model(
            model=config["model"],
            model_name=model_name,
            X_train=X_train,
            y_train=y_train,
            X_val=X_val,
            y_val=y_val,
            param_grid=None, # set to None to use Optuna
            search_method="optuna",  # Change to "random" for RandomizedSearchCV
            scoring=scoring,
            cv=cv
        )

## Compare Models

In [97]:
def make_eval_predictions(model, X_val):
    """Generate predictions and probabilities for evaluation."""
    y_pred = model.predict(X_val)
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    return y_pred, y_pred_proba

def produce_ROC_PR_curves(y_val, y_pred_proba):
    """Produce ROC and PR curves along with their metrics."""
    # ROC
    fpr, tpr, _ = metrics.roc_curve(y_val, y_pred_proba)
    roc_auc = metrics.auc(fpr, tpr)

    # PR
    precision, recall, _ = metrics.precision_recall_curve(y_val, y_pred_proba)
    average_precision = metrics.average_precision_score(y_val, y_pred_proba)
    baseline_precision = y_val.sum() / len(y_val)

    return (fpr, tpr, roc_auc), (precision, recall, average_precision, baseline_precision)


In [ ]:
# Logistic Regression (Single feature or basic version)
logreg_single = results['Logistic Regression (Single)']['model'].fit(X_train_single, y_train_single)
# Logistic Regression (Best Optuna regularization)
logreg_multi = results['Logistic Regression']['model'].fit(X_train, y_train)

# Random Forest (Optuna-tuned)
rf = results['Random Forest']['model'].fit(X_train, y_train)

# XGBoost (Optuna-tuned)
xgb = results['XGBoost']['model'].fit(X_train, y_train)

# XGBoost (selected-tuned)
best_xgb = XGBClassifier(
                max_depth=6,
                learning_rate=0.11483457172157868,
                n_estimators=127,
                subsample=0.9250144693067383,
                colsample_bytree=0.768549938469314,  # You had a formatting issue here; I assumed 0.9
                scale_pos_weight=8.893156628847267,
                reg_alpha=0.4254910148662459,
                reg_lambda=0.5370674064383636,
                # use_label_encoder=False,
                eval_metric="logloss",
                random_state=42
            ).fit(X_train, y_train)

In [103]:
# produce predictions and roc-pr values
models_to_evaluate = {
    "Logistic Regression (Single)": logreg_single,
    "Logistic Regression": logreg_multi,
    "Random Forest": rf,
    "XGBoost (Optuna)": xgb,
    "XGBoost (Selected)": best_xgb
}
eval_results = {}
for model_name, model in models_to_evaluate.items():
    if model_name == "Logistic Regression (Single)":
        y_pred, y_pred_proba = make_eval_predictions(model, X_val_single)
        (fpr, tpr, roc_auc), (precision, recall, avg_prec, base_prec) = produce_ROC_PR_curves(y_val_single, y_pred_proba)
    else:
        y_pred, y_pred_proba = make_eval_predictions(model, X_val)
        (fpr, tpr, roc_auc), (precision, recall, avg_prec, base_prec) = produce_ROC_PR_curves(y_val, y_pred_proba)

    eval_results[model_name] = {
        "y_pred": y_pred,
        "y_pred_proba": y_pred_proba,
        "roc": (fpr, tpr, roc_auc),
        "pr": (precision, recall, avg_prec, base_prec)
    }

### ROC - PR Curves

In [132]:
# Plot side-by-side
plt.style.use('seaborn-v0_8-talk')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 6), tight_layout=True)

### ROC Plot
ax1.plot([0, 1], [0, 1], 'k--', label='Random Guess', color='gray')
for model_name, res in eval_results.items():
    fpr, tpr, roc_auc = res['roc']
    ax1.plot(fpr, tpr, label=f'{model_name}\n(AUC = {roc_auc:.2f})')
# Inline labels
# ax1.text(0.5, 0.47, 'Random Guess', color='black', fontsize=10, rotation=43)

# Axes and annotations
ax1.set_xlim([0.0, 1.05])
ax1.set_ylim([0.0, 1.05])
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('ROC Curve')

ax1.text(0.4, 0.6, 'Better', fontsize=14, color='black', fontweight='bold')
ax1.text(0.25, 0.55, 'Worse', fontsize=14, color='red', fontweight='bold')
ax1.arrow(0.2, 0.6, 0.15, -0.2, head_width=0.02, head_length=0.02, fc='red', ec='red')
ax1.arrow(0.45, 0.5, -0.15, 0.2, head_width=0.02, head_length=0.02, fc='black', ec='black')

ax1.legend(loc='lower left', bbox_to_anchor=(-0.15, -0.47), ncol=2, fontsize=10)

### PR Plot
ax2.plot([0, 1], [eval_results[list(eval_results.keys())[0]]['pr'][3]]*2, '--', label='No Skill', color='gray')
for model_name, res in eval_results.items():
    precision, recall, avg_prec, base_prec = res['pr']
    ax2.plot(recall, precision, label=f'{model_name}\n(AP = {avg_prec:.2f})')

# add perfect model lines
ax2.axhline(1, color='black', linestyle='--')
ax2.axvline(1, color='black', linestyle='--')
# Inline labels
ax2.annotate("Perfect Model", xy=(0.75, 0.75), xycoords='data', fontsize=12, color='black', rotation=-45)

ax2.text(0.4, 0.6, 'Better', fontsize=14, color='black', fontweight='bold')
ax2.text(0.53, 0.57, 'Worse', fontsize=14, color='red', fontweight='bold')
ax2.arrow(0.55, 0.55, -0.15, -0.2, head_width=0.02, head_length=0.02, fc='red', ec='red')
ax2.arrow(0.25, 0.35, 0.15, 0.2, head_width=0.02, head_length=0.02, fc='black', ec='black')

# Axes and annotations
ax2.set_xlim([0.0, 1.05])
ax2.set_ylim([0.0, 1.05])
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')
ax2.set_title('Precision-Recall Curve')

ax2.legend(loc='lower left', bbox_to_anchor=(-0.15, -0.47), ncol=2, fontsize=10)

Precision-Recall is a useful measure of success of prediction when the classes are very imbalanced. In information retrieval, precision is a measure of the fraction of relevant items among actually returned items while recall is a measure of the fraction of items that were returned among all items that should have been returned. ‘Relevancy’ here refers to items that are postively labeled, i.e., true positives and false negatives.

Precision is defined as the number of true positives over the number of true positives plus the number of false positives.

Recall is defined as the number of true posistives over the number of true positives plus the number of false negatives.

The PR-curve shows the tradeoff between precision and recall for different thresholds. A high area under the curve represents both high recall and high precision. High precision is achieved by having few false positives in the returned results, and high recall is obtained by having few false negatives in the relevant results. High scores show that the classifier is returning accurate results and a majority of relevant results. 

A system with high recall but low precision returns most of the relevant items but the proportion of returned results that are incorrectly labeled is high.

### Comparing F1 and Average Precision across models

In [134]:
# Validation F1 scores
lr_single_val_f1 = results['Logistic Regression (Single)']['f1_score']
lr_val_f1 = results['Logistic Regression']['f1_score']
rf_val_f1 = results['Random Forest']['f1_score']
xgb_val_f1 = results['XGBoost']['f1_score']
xgb_best_val_f1 = metrics.f1_score(y_val, eval_results["XGBoost (Selected)"]["y_pred"])


# Validation Average Precision scores
lr_single_val_ap = results['Logistic Regression (Single)']['average_precision']
lr_val_ap = results['Logistic Regression']['average_precision']
rf_val_ap = results['Random Forest']['average_precision']
xgb_val_ap = results['XGBoost']['average_precision']
xbg_best_ap = metrics.average_precision_score(y_val, eval_results["XGBoost (Selected)"]["y_pred_proba"])

# offset scores under each label
labels = ['Wind Speed\nThreshold', 'Logistic\nRegression', 'Random\nForest', 'XGBoost', 'XGBoost\n(Selected)']
f1_scores = [lr_single_val_f1, lr_val_f1, rf_val_f1, xgb_val_f1, xgb_best_val_f1]
ap_scores = [lr_single_val_ap, lr_val_ap, rf_val_ap, xgb_val_ap, xbg_best_ap]
x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

# bar plot
fig, ax = plt.subplots(figsize=(6, 5))
ax.grid(axis='y', linestyle='--', alpha=0.5, zorder=10)
rects1 = ax.bar(x - width/2, f1_scores, width, label='F1 Score', color='blue')
rects2 = ax.bar(x + width/2, ap_scores, width, label='Average Precision', color='orange')
# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('F1 and Average Precision Scores by Model for Validation Set')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=0, ha='center', va='top')
ax.set_yticks(np.arange(0, 1.1, 0.1))

ax.set_ylim(0,1.05)
ax.legend()


## Selected Model Results

We chose XGBoost. This model will be used.

Can I just use the loaded ds to do this? Why do I need to do this individually?

### Functions

In [47]:
import numpy as np
import pandas as pd
from scipy.ndimage import label, find_objects
from typing import Tuple, List

In [51]:
# define function to calculate day of the water year
def get_day_of_water_year(date):
    # if input is type dt.date convert to dt.datetime
    if type(date) == dt.date:
        date = dt.datetime(date.year, date.month, date.day)
    # if the date is before october 1st, subtract one from the year
    if date.month < 10:
        year = date.year - 1
    else:
        year = date.year
    # create a datetime object for october 1st of the year
    oct_1 = dt.datetime(year, 10, 1)
    # calculate the day of water year
    day_of_water_year = (date - oct_1).days + 1
    return day_of_water_year


# ----------------------------------------------------------------------
# 2.  Build the base prediction DataFrame
# ----------------------------------------------------------------------
def build_prediction_df(predictions: np.ndarray,
                        time_index: pd.DatetimeIndex) -> pd.DataFrame:
    """
    Returns a DataFrame with temporal helper columns already attached.
    """
    df = pd.DataFrame(predictions, index=time_index,
                      columns=['sublimation_event'])

    df['day_of_water_year'] = df.index.map(get_day_of_water_year)
    df['month'] = df.index.month

    # Water‑year: Oct→Sep
    df['water_year'] = np.where(df.index.month < 10,
                                df.index.year,
                                df.index.year + 1)

    df['decade'] = (df['water_year'] // 10) * 10
    return df


# ----------------------------------------------------------------------
# 3.  Detect long events & annotate the DataFrame
# ----------------------------------------------------------------------
def label_long_events(df: pd.DataFrame,
                      col: str = 'sublimation_event',
                      min_len: int = 4
                      ) -> pd.DataFrame:
    """
    Adds two columns:
        • long_event_start : 1 on the first timestep of a long event
        • long_event       : 1 on every timestep inside a long event

    Parameters
    ----------
    df : DataFrame with a binary 'col' column
    col : column to search for runs of 1's
    min_len : minimum run length (in rows) to be called a 'long' event
    """
    labeled, _ = label(df[col].values == 1)
    slices = find_objects(labeled)

    df['long_event_start'] = 0
    df['long_event'] = 0

    for lbl, slc in enumerate(slices, start=1):
        if slc is None:
            continue
        run_len = slc[0].stop - slc[0].start
        if run_len >= min_len:
            df.iloc[slc[0].start, df.columns.get_loc('long_event_start')] = 1
            df.iloc[slc[0], df.columns.get_loc('long_event')] = 1
    return df


# ----------------------------------------------------------------------
# 4.  Summarise discrete long‑event counts
# ----------------------------------------------------------------------
def count_long_events(df: pd.DataFrame,
                      group_cols: Tuple[str, ...] = ('decade', 'water_year', 'month')
                      ) -> pd.DataFrame:
    """
    Returns a tidy table with one row per group and the number of
    discrete long events in `number_of_long_events`.
    """
    out = (
        df.groupby(list(group_cols))['long_event_start']
          .sum()
          .rename('number_of_long_events')
          .reset_index()
    )
    return out

### Results

<!-- # Apply to timeseries -->

In [ ]:
water_years = np.arange(1980,1982)
lat_sel = lat_sel_10_by_10
lon_sel = lon_sel_10_by_10
cleaned_df_list = []
result_df = pd.DataFrame(index=water_years, columns=['no_of_events', 'length_of_winter', 'percentage_of_events', 'sublimation_event_hours','average_wind_speed'])
monthly_results = pd.DataFrame(index=water_years, columns=['December', 'January', 'February', 'March'])
for water_year in water_years:
    print(f"Opening {water_year} data...")
    winter_slice = slice(f'{water_year-1}-12-01',f'{water_year}-03-31')
    # open reanalysis data
    tmp_wy1_ds = xr.open_dataset(f'/storage/dlhogan/data/raw_data/ERA5_reanalysis_western_NA_{water_year-1}_v2.nc')
    tmp_wy2_ds = xr.open_dataset(f'/storage/dlhogan/data/raw_data/ERA5_reanalysis_western_NA_{water_year}_v2.nc')
    print("Data opened... Performing temporal subset...")
    # temporal subset
    tmp_wy1_subset_ds = subset_ds(tmp_wy1_ds, winter_slice=winter_slice,)
    tmp_wy2_subset_ds = subset_ds(tmp_wy2_ds, winter_slice=winter_slice,)
    print("Temporal subset complete... Performing spatial subset...")
    # spatial subset
    tmp_wy1_subset_winter_ds = spatial_subset(tmp_wy1_subset_ds, lat_sel, lon_sel).mean(dim=['latitude', 'longitude'])
    tmp_wy2_subset_winter_2023 = spatial_subset(tmp_wy2_subset_ds, lat_sel, lon_sel).mean(dim=['latitude', 'longitude'])
    print("Spatial subset complete... Merging and cleaning data...")
    combined_df = create_df_combined(tmp_wy1_subset_ds, 
                                 tmp_wy2_subset_ds, 
                                 variables, 
                                 resample=True) 
    
    print("Data merged and cleaned...")
    cleaned_df_list[water_year] = combined_df

# Start here

In [57]:
# random binary time series for demonstration with length of 1000
np.random.seed(42)  # For reproducibility
prediction = np.random.randint(0, 2, size=100000)

# example temporal series
example_time_series = pd.date_range(start='1980-01-01', periods=len(prediction), freq='3H')

# 1) Build the full dataframe once
prediction_df = build_prediction_df(prediction, example_time_series)

# 2) Mark long events (≥4 consecutive 3‑h steps = ≥12 h here)
prediction_df = label_long_events(prediction_df, min_len=4)

# 3) Get counts (change group_cols as needed)
long_event_counts = count_long_events(
    prediction_df,
    group_cols=('decade', 'water_year', 'month')
)


In [65]:
prediction_df

In [82]:
import matplotlib.pyplot as plt
import seaborn as sns
# plot a timeseries of event hours for each year
sns.lineplot(
    data=(prediction_df.groupby('water_year')['long_event'].sum()*3).reset_index(),  # multiply by 3 to convert to hours
    x="water_year",
    y="long_event",
    estimator='sum',
    marker='o',
)




In [81]:
# plot a timeseries of event hours for each year
sns.lineplot(
    data=(prediction_df.groupby('day_of_water_year')['long_event'].sum()/8).reset_index(), # divide by 8 to get days
    x="day_of_water_year",
    y="long_event",
    estimator='mean',
    marker='o',
)

In [74]:
# plot a boxplot of the long event counts by month in each decade
sns.boxplot(
    data=long_event_counts,
    x="month",
    y="number_of_long_events",
    hue="decade",
    palette="tab10"
)

In [80]:
# plot the the mean distribution of long events across decades for each day of each wateryear
mean_long_events = prediction_df.groupby(['decade', 'month'])['long_event'].mean().reset_index()
mean_long_events_pivot = mean_long_events.pivot(index='month', columns ='decade', values='long_event')
plt.figure(figsize=(12, 6))
sns.lineplot(data=mean_long_events_pivot, dashes=False)

### Relationship between primary variable and results

In [ ]:
plt.scatter(result_df['no_of_events'], result_df['average_wind_speed'])
# add the r2 between variables
# slope, intercept, r_value, p_value, std_err = linregress(result_df['no_of_events'], result_df['average_wind_speed'])
# plt.plot(result_df['no_of_events'], intercept + slope * result_df['no_of_events'], 'r', label='fitted line')

### Summary results

! Save the data that goes into it so I can produce this figure in my results section.

In [ ]:
plt.style.use('seaborn-v0_8-talk')

fig = plt.figure()

# set height ratios for subplots
gs = gridspec.GridSpec(2, 1, height_ratios=[2, 2]) 

# the first subplot
ax0 = plt.subplot(gs[0])

ax0.plot(result_df.index, result_df['no_of_events']*3, '-o', color='blue', label='Duration of Sublimation Events', markersize=5)
ax0.scatter(x=[2022, 2023],
            y=[len(w22_large_sublimation_long_events)*3, len(w23_large_sublimation_long_events)*3],
            color='red', s=100, label='Observed')
ax0.set_ylabel('Duration of Sublimation\nEvents (hours)')
# add horizontal grid
ax0.grid(axis='y', linestyle='--', alpha=0.7)
ax0.set_ylim(0,450)

# annotate observed values
ax0.annotate('Observed', xy=(2022, len(w22_large_sublimation_long_events)*3), xytext=(2018, len(w22_large_sublimation_long_events)*3+75),
             arrowprops=dict(facecolor='black', arrowstyle='->', linewidth=2), fontsize=18, color='black')
ax0.annotate('Observed', xy=(2023, len(w23_large_sublimation_long_events)*3), xytext=(2018, len(w22_large_sublimation_long_events)*3+75),
                arrowprops=dict(facecolor='black', arrowstyle='->', linewidth=2), fontsize=18, color='black')

# annotate the line 
ax0.annotate('Modeled\nSublimation\nEvents', xy=(2024, (result_df['no_of_events']*3).iloc[-1]), 
             xytext=(2025, (result_df['no_of_events']*3).iloc[-1]), 
             fontsize=18, 
             va='center',
             color='blue')

# remove top and right spine
ax0.spines['top'].set_visible(False)
ax0.spines['right'].set_visible(False)

ax1 = plt.subplot(gs[1], sharex = ax0)  
ax1.plot(result_df.index, result_df['average_wind_speed'], '-o', color='darkorange', label='Average Winter 500-mb Wind Speed', markersize=5)
ax1.plot(guc_obs.loc[1980:2024].index, guc_obs.loc[1980:2024]['winds'], '-o', color='black', label='Observed GJT Wind Speed', markersize=5)
ax1.set_xlabel('Water Year')
ax1.set_ylabel('Average Winter\nWind Speed (m/s)')

# annotate the line
ax1.annotate('ERA5 500-mb \nWind Speed', xy=(2024, result_df['average_wind_speed'].iloc[-1]),
                xytext=(2025, result_df['average_wind_speed'].iloc[-1]-1), 
                fontsize=18, 
                va='center',
                color='darkorange')
ax1.annotate('GJT 500-mb \nWind Speed', xy=(2024, result_df['average_wind_speed'].iloc[-2]),
                xytext=(2025, result_df['average_wind_speed'].iloc[-2]), 
                fontsize=18, 
                va='center',
                color='black')

# add horizontal grid
ax1.grid(axis='y', linestyle='--', alpha=0.7)
ax1.set_ylim(0,23)

# remove right spine
ax1.spines['right'].set_visible(False)

# show every 5 years on the x-axis
ax1.set_xticks(np.arange(1980, 2025, 5))
ax1.set_xticklabels(np.arange(1980, 2025, 5), fontsize=14)


# remove vertical gap between subplots
plt.subplots_adjust(hspace=.0)
plt.show()

#### Look for trends

In [ ]:
fig, axs = plt.subplots(2,2, tight_layout=True);
axs = axs.ravel()
cols = ['no_of_events', 'average_wind_speed', 'percentage_of_events', 'sublimation_event_hours']

for i in range(len(axs)):
    result_df[cols[i]].hist(ax=axs[i], color='blue', bins=8)
    axs[i].set_title(cols[i])

In [ ]:
# perform the Mann-Kendall test
mk_result_no_events = mk.original_test(result_df['no_of_events'], alpha=0.05)
mk_gjt_obs = mk.original_test(guc_obs['winds'], alpha=0.05)
mk_result_average_windspeed = mk.original_test(result_df['average_wind_speed'], alpha=0.05)
mk_result_percentage_of_events = mk.original_test(result_df['percentage_of_events'], alpha=0.05)

print(mk_result_no_events)
print(mk_gjt_obs)
print(mk_result_average_windspeed)
print(mk_result_percentage_of_events)


### Seasonal Boxplots

In [310]:
 # plot boxplot of monthly_results
fig, ax = plt.subplots(figsize=(6, 6))
ax.boxplot([monthly_results['December']*100, monthly_results['January']*100, monthly_results['February']*100, monthly_results['March']*100],
              labels=['December', 'January', 'February', 'March'])
ax.set_title('Monthly Sublimation Events Distribution')
ax.set_ylabel("Monthly sublimation event frequency (%)")
ax.set_xlabel('Month')

### Decadal comparison boxplots

## Appendix

#### Example figures...
To demonstrate how this cross-validation works

In [ ]:
rng = np.random.RandomState(1338)
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm
n_splits = 5
# Generate the class/group data
n_points = 100
X = rng.randn(100, 10)

percentiles_classes = [0.1, 0.9]
y = np.hstack([[ii] * int(100 * perc) for ii, perc in enumerate(percentiles_classes)])

# Generate uneven groups
group_prior = rng.dirichlet([2] * 10)
groups = np.repeat(np.arange(10), rng.multinomial(100, group_prior))

In [ ]:
def plot_cv_indices(cv, X, y, group, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""
    use_groups = "Group" in type(cv).__name__
    groups = group if use_groups else None
    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y, groups=groups)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )

    # ax.scatter(
    #     range(len(X)), [ii + 2.5] * len(X), c=group, marker="_", lw=lw, cmap=cmap_data
    # )

    # Formatting
    yticklabels = list(range(n_splits)) + ["class", "group"]
    ax.set(
        yticks=np.arange(n_splits + 2) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 2.2, -0.2],
        xlim=[0, 100],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax

In [ ]:
cvs = [StratifiedShuffleSplit, KFold, StratifiedKFold]

for cv in cvs:
    fig, ax = plt.subplots(figsize=(6, 3))
    
    if cv == StratifiedKFold:
        plot_cv_indices(cv(n_splits, shuffle=True, random_state=42), X, y, groups, ax, n_splits)
    else:
        plot_cv_indices(cv(n_splits), X, y, groups, ax, n_splits)
    ax.legend(
        [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
        ["Testing set", "Training set"],
        loc=(1.02, 0.8),
    )
    # Make the legend fit
    plt.tight_layout()
    fig.subplots_adjust(right=0.7)

### Model 1: Logistic Regression (Single variable)

In [ ]:
lr_single = LogisticRegression(class_weight='balanced', random_state=42, solver='lbfgs', penalty='l2', C=0.5)
# GridSearchCV parameters to test
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = np.arange(0.01, 0.5, 0.01)

# define the grid
param_grid = {
    # 'solver': solvers,
    # 'penalty': penalty,
    'C': c_values,
}

In [ ]:
# create the gridsearch
grid_search = GridSearchCV(
    estimator=lr_single, # our logistic regression model
    param_grid=param_grid, # the grid of parameters to test
    scoring=scoring, # the scoring metrics to use
    refit = 'F1', # the metric to use to select the best model
    cv=cv, # the cross-validation object
    n_jobs=4, # use 4 cores
    return_train_score=True, # return the training score
)

In [ ]:
# run the model
results_lr = grid_search.fit(X_train_single, y_train_single)
results_lr_cv = results_lr.cv_results_

#### Model Performance

In [ ]:
plt.figure(figsize=(5, 5))
plt.title("GridSearchCV evaluating using multiple scorers simultaneously", fontsize=16)

plt.xlabel("min_samples_split")
plt.ylabel("Score")

ax = plt.gca()
ax.set_xlim(min(results_lr_cv[f"param_{list(param_grid.keys())[0]}"]), max(results_lr_cv[f"param_{list(param_grid.keys())[0]}"]))
ax.set_ylim(0, 1)

# Get the regular numpy array from the MaskedArray
X_axis = np.array(results_lr_cv[f"param_{list(param_grid.keys())[0]}"].data, dtype=float)

for scorer, color in zip(sorted(scoring), ["g", "k"]):
    for sample, style in (("train", "--"), ("test", "-")):
        sample_score_mean = results_lr_cv["mean_%s_%s" % (sample, scorer)]
        sample_score_std = results_lr_cv["std_%s_%s" % (sample, scorer)]
        ax.fill_between(
            X_axis,
            sample_score_mean - sample_score_std,
            sample_score_mean + sample_score_std,
            alpha=0.1 if sample == "test" else 0,
            color=color,
        )
        ax.plot(
            X_axis,
            sample_score_mean,
            style,
            color=color,
            alpha=1 if sample == "test" else 0.7,
            label="%s (%s)" % (scorer, sample),
        )

    best_index = np.nonzero(results_lr_cv["rank_test_%s" % scorer] == 1)[0][0]
    best_score = results_lr_cv["mean_test_%s" % scorer][best_index]

    # Plot a dotted vertical line at the best score for that scorer marked by x
    ax.plot(
        [
            X_axis[best_index],
        ]
        * 2,
        [0, best_score],
        linestyle="-.",
        color=color,
        marker="x",
        markeredgewidth=3,
        ms=8,
    )

    # Annotate the best score for that scorer
    ax.annotate("%0.2f" % best_score, (X_axis[best_index], best_score + 0.005))
    # Annotate the best parameter for that scorer
    ax.annotate(
        "%s" % round(X_axis[best_index],1),
        (X_axis[best_index], -0.01),
        ha="center",
        va="top",
    )

plt.legend(loc="best")
plt.grid(False)
plt.show()

In [ ]:
print("Best: %f using %s" % (results_lr.best_score_, results_lr.best_params_))
means = results_lr_cv['mean_test_F1']
stds = results_lr_cv['std_test_F1']
params = results_lr_cv['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#### Fit the best model and classify results

In [ ]:
# Fit the model
lr_single_best = LogisticRegression(class_weight='balanced', 
                             random_state=42, 
                             solver='lbfgs', 
                             penalty='l2', 
                             C=0.03).fit(X_train_single, y_train_single)
lr_single_y_pred_proba = lr_single_best.predict_proba(X_val_single)[:, 1]

# ROC
lr_single_fpr, lr_single_tpr, _ = metrics.roc_curve(y_val_single, lr_single_y_pred_proba)
lr_single_roc_auc = metrics.auc(lr_single_fpr, lr_single_tpr)

# PR
lr_single_precision, lr_single_recall, _ = metrics.precision_recall_curve(y_val_single, lr_single_y_pred_proba)
lr_single_average_precision = metrics.average_precision_score(y_val_single, lr_single_y_pred_proba)
lr_single_baseline_precision = y_val_single.sum() / len(y_val_single)

# show confusion matrix
metrics.ConfusionMatrixDisplay.from_estimator(lr_single_best, X_val_single, y_val_single, cmap='Blues', colorbar=False, display_labels=['non-event', 'event'])

### Model 2: Multi-variable Logistic Regressor

Some background. Logistic "regression" is a classification algorithm that is used to predict the *probability* of a categorical dependent value. It sounds similar to linear regression in some ways, but it is distinctly different because the dependent and independent variables are not related linearly, instead the linear relationship is between the independent variables and the logit (log of odds) of independent variable, essentially its probability. The method can then be used to predict the probability of occurrence of a binary event utilizing a logit function (like Sigmoid/logistic function)
$$p = \frac{1}{1+e^{-y}}$$

where $y = \Beta_0 +\Beta_1X_1 +\Beta_2X_2 ...+\Beta_nX_n$

Logistic regression uses maximum liklihood estimation approach. MLE is a liklihood maximation method while OLS is a distance-minimizing method. Maximizing the liklihood function determines parameters that are most likely to produce the observed data

Model choices include:
- not standardized (insensitive to magnitude)
- 80/20 train test split for validation
- cross validation on k=10 folds with stratified shuffle split.
- performance is evaluated using f1-score, 

In [ ]:
lr = LogisticRegression(class_weight='balanced', random_state=42, solver='lbfgs', penalty='l2', C=0.5)

In [ ]:
# GridSearchCV parameters to test
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = np.arange(0.01, 0.5, 0.01)

# define the grid
param_grid = {
    # 'solver': solvers,
    # 'penalty': penalty,
    'C': c_values,
}

In [ ]:
# create the gridsearch
grid_search = GridSearchCV(
    estimator=lr, # our logistic regression model
    param_grid=param_grid, # the grid of parameters to test
    scoring=scoring, # the scoring metrics to use
    refit = 'F1', # the metric to use to select the best model
    cv=cv, # the cross-validation object
    n_jobs=4, # use 4 cores
    return_train_score=True, # return the training score
)

In [ ]:
# run the model
results_lr = grid_search.fit(X_train, y_train)
results_lr_cv = results_lr.cv_results_

#### Model Performance

In [ ]:
plt.figure(figsize=(5, 5))
plt.title("GridSearchCV evaluating using multiple scorers simultaneously", fontsize=16)

plt.xlabel("min_samples_split")
plt.ylabel("Score")

ax = plt.gca()
ax.set_xlim(min(results_lr_cv[f"param_{list(param_grid.keys())[0]}"]), max(results_lr_cv[f"param_{list(param_grid.keys())[0]}"]))
ax.set_ylim(0, 1)

# Get the regular numpy array from the MaskedArray
X_axis = np.array(results_lr_cv[f"param_{list(param_grid.keys())[0]}"].data, dtype=float)

for scorer, color in zip(sorted(scoring), ["g", "k"]):
    for sample, style in (("train", "--"), ("test", "-")):
        sample_score_mean = results_lr_cv["mean_%s_%s" % (sample, scorer)]
        sample_score_std = results_lr_cv["std_%s_%s" % (sample, scorer)]
        ax.fill_between(
            X_axis,
            sample_score_mean - sample_score_std,
            sample_score_mean + sample_score_std,
            alpha=0.1 if sample == "test" else 0,
            color=color,
        )
        ax.plot(
            X_axis,
            sample_score_mean,
            style,
            color=color,
            alpha=1 if sample == "test" else 0.7,
            label="%s (%s)" % (scorer, sample),
        )

    best_index = np.nonzero(results_lr_cv["rank_test_%s" % scorer] == 1)[0][0]
    best_score = results_lr_cv["mean_test_%s" % scorer][best_index]

    # Plot a dotted vertical line at the best score for that scorer marked by x
    ax.plot(
        [
            X_axis[best_index],
        ]
        * 2,
        [0, best_score],
        linestyle="-.",
        color=color,
        marker="x",
        markeredgewidth=3,
        ms=8,
    )

    # Annotate the best score for that scorer
    ax.annotate("%0.2f" % best_score, (X_axis[best_index], best_score + 0.005))
    # Annotate the best parameter for that scorer
    ax.annotate(
        "%s" % round(X_axis[best_index],1),
        (X_axis[best_index], -0.01),
        ha="center",
        va="top",
    )

plt.legend(loc="best")
plt.grid(False)
plt.show()

In [ ]:
print("Best: %f using %s" % (results_lr.best_score_, results_lr.best_params_))
means = results_lr_cv['mean_test_F1']
stds = results_lr_cv['std_test_F1']
params = results_lr_cv['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#### Fit the best model and classify results

In [ ]:
# Fit the model
lr_best = LogisticRegression(class_weight='balanced', 
                             random_state=42, 
                             solver='lbfgs', 
                             penalty='l2', 
                             C=0.03).fit(X_train, y_train)
lr_y_pred_proba = lr_best.predict_proba(X_val)[:, 1]

# ROC
lr_fpr, lr_tpr, _ = metrics.roc_curve(y_val, lr_y_pred_proba)
lr_roc_auc = metrics.auc(lr_fpr, lr_tpr)

# PR
lr_precision, lr_recall, _ = metrics.precision_recall_curve(y_val, lr_y_pred_proba)
lr_average_precision = metrics.average_precision_score(y_val, lr_y_pred_proba)
lr_baseline_precision = y_val.sum() / len(y_val)

# show confusion matrix
metrics.ConfusionMatrixDisplay.from_estimator(lr_best, X_val, y_val, cmap='Blues', colorbar=False, display_labels=['non-event', 'event'])

### Model 3: Random Forest Classifier

| Model Type             | max_depth | max_features | n_estimators | min_samples_split | min_samples_leaf | Mean F1-Score |
|------------------------|-----------|---------------|---------------|--------------------|-------------------|----------------|
| Areal Average          | 7         | 3             | 20            | 10                 | —                 | 0.500          |
| Closest Point          | 10        | 4             | 60            | 5                  | 4                 | 0.520          |
| 5-Grid Areal Average   | 10        | 4             | —             | 2                  | 1                 | 0.545 (±0.109) |
| 17×17 Grid             | 15        | 4             | 5             | 2                  | 2                 | 0.589          |


In [ ]:
rf = RandomForestClassifier(random_state=42, 
                            max_depth=15, 
                            max_features=4,
                            n_estimators=30, 
                            min_samples_split=2,
                            min_samples_leaf=2,
                            )
run_description=input("Enter a description for the run: ")
# define the grid
param_grid = {
    'n_estimators': [5, 10, 20, 30, 40, 50,100], # number of trees in the forest
    # 'max_depth': [5,7,10,15], # maximum depth of the tree, balances overfitting and underfitting
    # 'min_samples_split': [2, 5, 10,15,20], # minimum number of samples required to split an internal node
    # 'min_samples_leaf': [1, 2, 4, 5, 10, 15, 20], # minimum number of samples required to be at a leaf node
    # 'max_features': [2,3,4,5], # number of features to consider when looking for the best split
}

# scoring metrics
scoring = {"F1":"f1", 
           "Average_Precision":metrics.make_scorer(metrics.average_precision_score),
        #    "Recall":metrics.make_scorer(metrics.recall_score),
        #    "Precision":metrics.make_scorer(metrics.precision_score),
           }

In [ ]:
rf_grid_search = GridSearchCV(
    estimator=rf, # our logistic regression model
    param_grid=param_grid, # the grid of parameters to test
    scoring=scoring, # the scoring metrics to use
    refit = 'F1', # the metric to use to select the best model
    cv=cv, # the cross-validation object
    n_jobs=4, # use 4 cores
    return_train_score=True, # return the training score
)
results_rf = rf_grid_search.fit(X_train, y_train)
results_rf_cv = results_rf.cv_results_

#### Model Performance

In [ ]:
plt.figure(figsize=(5, 5))
plt.title("GridSearchCV evaluating using multiple scorers simultaneously", fontsize=16)

plt.xlabel(list(param_grid.keys())[0])
plt.ylabel("Score")

ax = plt.gca()
ax.set_xlim(min(results_rf_cv[f"param_{list(param_grid.keys())[0]}"]), max(results_rf_cv[f"param_{list(param_grid.keys())[0]}"]))
ax.set_ylim(0, 1)

# Get the regular numpy array from the MaskedArray
X_axis = np.array(results_rf_cv[f"param_{list(param_grid.keys())[0]}"].data, dtype=float)

for scorer, color in zip(sorted(scoring), ["g", "k", ]):
    for sample, style in (("train", "--"), ("test", "-")):
        sample_score_mean = results_rf_cv["mean_%s_%s" % (sample, scorer)]
        sample_score_std = results_rf_cv["std_%s_%s" % (sample, scorer)]
        ax.fill_between(
            X_axis,
            sample_score_mean - sample_score_std,
            sample_score_mean + sample_score_std,
            alpha=0.1 if sample == "test" else 0,
            color=color,
        )
        ax.plot(
            X_axis,
            sample_score_mean,
            style,
            color=color,
            alpha=1 if sample == "test" else 0.7,
            label="%s (%s)" % (scorer, sample),
        )

    best_index = np.nonzero(results_rf_cv["rank_test_%s" % scorer] == 1)[0][0]
    best_score = results_rf_cv["mean_test_%s" % scorer][best_index]

    # Plot a dotted vertical line at the best score for that scorer marked by x
    ax.plot(
        [
            X_axis[best_index],
        ]
        * 2,
        [0, best_score],
        linestyle="-.",
        color=color,
        marker="x",
        markeredgewidth=3,
        ms=8,
    )

    # Annotate the best score for that scorer
    ax.annotate("%0.2f" % best_score, (X_axis[best_index], best_score + 0.005))
    # Annotate the best parameter for that scorer
    ax.annotate(
        "%s" % round(X_axis[best_index],1),
        (X_axis[best_index], -0.01),
        ha="center",
        va="top",
    )

plt.legend(loc="best")
plt.grid(False)
plt.show()

In [ ]:
print("Best: %f using %s" % (results_rf.best_score_, results_rf.best_params_))
means = results_rf_cv['mean_test_F1']
stds = results_rf_cv['std_test_F1']
params = results_rf_cv['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#### Log model results

In [ ]:
def log_rf_model_run(
    run_description,
    model_type,
    f1_score,
    n_estimators=None,
    max_depth=None,
    min_samples_split=None,
    min_samples_leaf=None,
    max_features=None,
    output_csv="rf_model_run_log.csv"
):
    """
    Logs the summary of a Random Forest model run to a CSV file.

    Parameters:
        run_description (str): Brief explanation of the uniqueness of this model run.
        model_type (str): The general type or configuration name of the model (e.g., 'areal average', 'closest point').
        f1_score (float): The mean F1-score for this model run.
        All other parameters are optional hyperparameters used in the run.
        output_csv (str): Path to the output CSV file (default: "rf_model_run_log.csv").
    """

    row = pd.DataFrame([{
        "run_description": run_description,
        "model_type": model_type,
        "f1_score": f1_score,
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf,
        "max_features": max_features
    }])

    if os.path.exists(output_csv):
        row.to_csv(output_csv, mode='a', header=False, index=False)
    else:
        row.to_csv(output_csv, mode='w', header=True, index=False)

    print(f"✅ Random Forest model run logged successfully to: {output_csv}")


In [ ]:
best_f1_score = results_rf.best_score_
best_params = results_rf.best_params_

if 'n_estimators' not in best_params.keys():
    best_params['n_estimators'] = rf.n_estimators
if 'max_depth' not in best_params.keys():
    best_params['max_depth'] = rf.max_depth
if 'min_samples_split' not in best_params.keys():
    best_params['min_samples_split'] = rf.min_samples_split
if 'min_samples_leaf' not in best_params.keys():
    best_params['min_samples_leaf'] = rf.min_samples_leaf
if 'max_features' not in best_params.keys():
    best_params['max_features'] = rf.max_features

log_rf_model_run(
    run_description=run_description,
    model_type="Areal Average",
    f1_score=best_f1_score,
    n_estimators=best_params.get('n_estimators'),
    max_depth=best_params.get('max_depth'),
    min_samples_split=best_params.get('min_samples_split'),
    min_samples_leaf=best_params.get('min_samples_leaf'),
    max_features=best_params.get('max_features')
)


#### Feature Importance

In [ ]:
important_features = pd.Series(data=results_rf.best_estimator_.feature_importances_,index=data_X.columns)
important_features.sort_values(ascending=False,inplace=True)

# plot
plt.figure(figsize=(10, 6))
important_features.plot(kind='bar')
plt.title('Feature Importance')

#### Fit the best model

In [ ]:
# Fit the model
rf_best = RandomForestClassifier(random_state=42, 
                            n_estimators=best_params.get('n_estimators'),
                            max_depth=best_params.get('max_depth'),
                            min_samples_split=best_params.get('min_samples_split'),
                            min_samples_leaf=best_params.get('min_samples_leaf'),
                            max_features=best_params.get('max_features')
                            ).fit(X_train, y_train)
                                 
rf_y_pred_proba = rf_best.predict_proba(X_val)[:, 1]

# ROC
rf_fpr, rf_tpr, _ = metrics.roc_curve(y_val, rf_y_pred_proba)
rf_roc_auc = metrics.auc(rf_fpr, rf_tpr)

# PR
rf_precision, rf_recall, _ = metrics.precision_recall_curve(y_val, rf_y_pred_proba)
rf_average_precision = metrics.average_precision_score(y_val, rf_y_pred_proba)
rf_baseline_precision = y_val.sum() / len(y_val)

# Show confusion matrix
metrics.ConfusionMatrixDisplay.from_estimator(rf_best, X_val, y_val, cmap='Blues', colorbar=False, display_labels=['non-event', 'event'])
# print f1 score
print("F1 Score: ", metrics.f1_score(y_val, rf_best.predict(X_val)))
# print average precision score
print(f"AP: {metrics.average_precision_score(y_val, rf_best.predict(X_val))}")

### Model 4: XGBoost Classifier

| Model Type           | n_estimators | learning_rate | max_depth | scale_pos_weight | subsample | Mean F1-Score |
|----------------------|--------------|----------------|-----------|-------------------|------------|----------------|
| Areal Average        | 30           | 0.3            | 7         | 10                | 0.3        | 0.640          |
| Closest Point        | —            | 0.7            | 7         | 5                 | 0.7        | 0.571          |
| 5-Grid Average       | 30           | 0.2            | 10        | 8                 | 0.5        | 0.590          |
| 10-Grid Average      | 60           | 0.1            | 10        | 5                 | 0.7        | 0.610          |
| 20-Grid Average      | —            | 0.2            | 7         | 10                | 0.5        | 0.660          |
| 17×17 Grid           | 80           | 0.5            | 7         | 50                | 0.9        | 0.630          |


In [ ]:
xgb = XGBClassifier(random_state=42, 
                                   n_estimators=30, 
                                   scale_pos_weight=20, 
                                   learning_rate=0.3,
                                   max_depth=7,
                                   subsample= 0.9,
                                   objective="binary:logistic"
                                )
# Add a short description of what makes this run unique
run_description = input("Enter a brief description of this XGBoost model run: ")
model_area = input("Enter the model area (e.g., '10-grid Areal Average', 'Closest Point'): ")

default = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

# Define the range of scale_pos_weight ratios to test
ratios = [5, 10, 20, 50, 100]
# define the grid
param_grid = {
    'learning_rate':  [0.1, 0.2, 0.3,0.5,0.7], # st controls the step size at each boosting iteration
   # 'n_estimators': np.arange(10,50,10), # number of trees in the forest
   #  'max_depth': [2,3,5,7,10], # controls the maximum depth of the decision trees used in the model
    'scale_pos_weight': ratios, # controls the balance of positive and negative weights
    'subsample': [0.1,0.3, 0.5, 0.7, 0.9, 1.], # fraction of samples to be used for each tree
}

# scoring metrics
scoring = {"F1":"f1", 
           "Average_Precision":metrics.make_scorer(metrics.average_precision_score),
        #    "Recall":metrics.make_scorer(metrics.recall_score),
        #    "Precision":metrics.make_scorer(metrics.precision_score),
           }

In [ ]:
xgb_grid_search = GridSearchCV(
    estimator=xgb, # our logistic regression model
    param_grid=param_grid, # the grid of parameters to test
    scoring=scoring, # the scoring metrics to use
    refit = 'F1', # the metric to use to select the best model
    cv=cv, # the cross-validation object
    n_jobs=4, # use 4 cores
    return_train_score=True, # return the training score
)
results_xgb = xgb_grid_search.fit(X_train, y_train)
results_xgb_cv = results_xgb.cv_results_



#### Log model results

In [ ]:
def log_model_run(
    run_description,
    model_type,
    f1_score,
    n_estimators=None,
    learning_rate=None,
    max_depth=None,
    scale_pos_weight=None,
    subsample=None,
    min_samples_split=None,
    min_samples_leaf=None,
    max_features=None,
    output_csv="model_run_log.csv"
):
    """
    Logs the summary of a model run to a CSV file.

    Parameters:
        run_description (str): Brief explanation of the uniqueness of this model run.
        model_type (str): The general type or configuration name of the model (e.g., 'areal average', 'closest point').
        f1_score (float): The mean F1-score for this model run.
        All other parameters are optional hyperparameters used in the run.
        output_csv (str): Path to the output CSV file (default: "model_run_log.csv").
    """

    # Build a single-row DataFrame
    row = pd.DataFrame([{
        "run_description": run_description,
        "model_type": model_type,
        "f1_score": f1_score,
        "n_estimators": n_estimators,
        "learning_rate": learning_rate,
        "max_depth": max_depth,
        "scale_pos_weight": scale_pos_weight,
        "subsample": subsample,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf,
        "max_features": max_features
    }])

    # Append to CSV, creating file if it doesn't exist
    if os.path.exists(output_csv):
        row.to_csv(output_csv, mode='a', header=False, index=False)
    else:
        row.to_csv(output_csv, mode='w', header=True, index=False)

    print(f"✅ Model run logged successfully: {output_csv}")


In [ ]:
# Assuming you've already run a GridSearchCV or RandomizedSearchCV:
best_f1_score = results_xgb.best_score_
best_params = results_xgb.best_params_

if 'n_estimators' not in best_params.keys():
    best_params['n_estimators'] = xgb.n_estimators
if 'learning_rate' not in best_params.keys():
    best_params['learning_rate'] = xgb.learning_rate
if 'max_depth' not in best_params.keys():
    best_params['max_depth'] = xgb.max_depth
if 'scale_pos_weight' not in best_params.keys():
    best_params['scale_pos_weight'] = xgb.scale_pos_weight
if 'subsample' not in best_params.keys():
    best_params['subsample'] = xgb.subsample
if 'max_features' not in best_params.keys():
    best_params['max_features'] = xgb.max_features if hasattr(xgb, 'max_features') else None  # optional; only include if used

# Now call the log function
log_model_run(
    run_description=run_description,
    model_type=model_area,
    f1_score=best_f1_score,
    n_estimators=best_params.get('n_estimators'),
    learning_rate=best_params.get('learning_rate'),
    max_depth=best_params.get('max_depth'),
    scale_pos_weight=best_params.get('scale_pos_weight'),
    subsample=best_params.get('subsample'),
    max_features=best_params.get('max_features')  # optional; only include if used
)


#### Model Performance

In [ ]:
print("Best: %f using %s" % (results_xgb.best_score_, results_xgb.best_params_))
means = results_xgb_cv['mean_test_F1']
stds = results_xgb_cv['std_test_F1']
params = results_xgb_cv['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
if len(param_grid.keys()) == 1:
    # Plot the results for a single parameter grid
    plt.figure(figsize=(5, 5))
    plt.title("GridSearchCV evaluating using multiple scorers simultaneously", fontsize=16)

    plt.xlabel(list(param_grid.keys())[0])
    plt.ylabel("Score")

    ax = plt.gca()
    ax.set_xlim(min(results_xgb_cv[f"param_{list(param_grid.keys())[0]}"]), max(results_xgb_cv[f"param_{list(param_grid.keys())[0]}"]))
    ax.set_ylim(0, 1)

    # Get the regular numpy array from the MaskedArray
    X_axis = np.array(results_xgb_cv[f"param_{list(param_grid.keys())[0]}"].data, dtype=float)

    for scorer, color in zip(sorted(scoring), ["g", "k", ]):
        for sample, style in (("train", "--"), ("test", "-")):
            sample_score_mean = results_xgb_cv["mean_%s_%s" % (sample, scorer)]
            sample_score_std = results_xgb_cv["std_%s_%s" % (sample, scorer)]
            ax.fill_between(
                X_axis,
                sample_score_mean - sample_score_std,
                sample_score_mean + sample_score_std,
                alpha=0.1 if sample == "test" else 0,
                color=color,
            )
            ax.plot(
                X_axis,
                sample_score_mean,
                style,
                color=color,
                alpha=1 if sample == "test" else 0.7,
                label="%s (%s)" % (scorer, sample),
            )

        best_index = np.nonzero(results_xgb_cv["rank_test_%s" % scorer] == 1)[0][0]
        best_score = results_xgb_cv["mean_test_%s" % scorer][best_index]

        # Plot a dotted vertical line at the best score for that scorer marked by x
        ax.plot(
            [
                X_axis[best_index],
            ]
            * 2,
            [0, best_score],
            linestyle="-.",
            color=color,
            marker="x",
            markeredgewidth=3,
            ms=8,
        )

        # Annotate the best score for that scorer
        ax.annotate("%0.2f" % best_score, (X_axis[best_index], best_score + 0.005))
        # Annotate the best parameter for that scorer
        ax.annotate(
            "%s" % round(X_axis[best_index],1),
            (X_axis[best_index], 0.1),
            ha="center",
            va="top",
        )

    plt.legend(loc="best")
    plt.grid(False)
    plt.show()

#### Feature Importance

In [ ]:
important_features = pd.Series(data=results_xgb.best_estimator_.feature_importances_,index=data_X.columns)
important_features.sort_values(ascending=False,inplace=True)

# plot
plt.figure(figsize=(5, 5))
important_features.plot(kind='bar')
plt.title('Feature Importance')

#### Fit the best model and classify results

In [ ]:
xgb_best = XGBClassifier(random_state=42, 
               n_estimators=best_params.get('n_estimators'),
               learning_rate=best_params.get('learning_rate'),
               max_depth=best_params.get('max_depth'),
               scale_pos_weight=best_params.get('scale_pos_weight'),
               subsample=best_params.get('subsample'),
               objective="binary:logistic").fit(X_train, y_train)

In [ ]:
# Fit the model 

y_pred_proba = xgb_best.predict_proba(X_val)[:, 1]

# ROC
xgb_fpr, xgb_tpr, _ = metrics.roc_curve(y_val, y_pred_proba)
xgb_roc_auc = metrics.auc(xgb_fpr, xgb_tpr)

# PR
xgb_precision, xgb_recall, _ = metrics.precision_recall_curve(y_val, y_pred_proba)
xgb_average_precision = metrics.average_precision_score(y_val, y_pred_proba)
xgb_baseline_precision = y_val.sum() / len(y_val)

# # Show confusion matrix
# metrics.ConfusionMatrixDisplay.from_estimator(xgb_best, X_val, y_val, cmap='Blues', colorbar=False, display_labels=['non-event', 'event'])
# # print f1 score
# print("F1 Score: ", metrics.f1_score(y_val, xgb_best.predict(X_val)))
# # print average precision score
# print(f"AP: {metrics.average_precision_score(y_val, xgb_best.predict(X_val))}")

### Plot with plotly

In [ ]:
# plotting
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# make plotly work 
init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=result_df.index,
    y=result_df['no_of_events']*3,
    mode='lines+markers',
    name='Percentage of Events',
    line=dict(color='blue', width=2),   
), secondary_y=False,)
fig.add_trace(go.Scatter(
    x=[2022, 2023],
    y=[len(w22_large_sublimation_long_events)*3, len(w23_large_sublimation_long_events)*3],
    mode='markers',
    name='Observed',
    marker=dict(color='red', size=10)
), secondary_y=False,)  

# add secondary axis with average wind speed
fig.add_trace(go.Scatter(
    x=result_df.index,
    y=result_df['average_wind_speed'],
    mode='lines+markers',
    name='Average Wind Speed',
    line=dict(color='orange', width=2),\
), secondary_y=True,)

fig.update_layout(
    width=1000,
    height=400,
    title='Total Number of Hours with Sublimation Events by Winter',
    xaxis_title='Water Year',
    yaxis_title='Hours',
    yaxis2_title='Average Wind Speed (m/s)',
)